In [1]:
import Distributions; dist = Distributions
import Random
import Statistics; stats = Statistics

Statistics

In [2]:
# Price of MBS
# if t< tau < T
# Mt = discounted c + discounted P_tau 
# if T <= tau
# Mt = 

In [3]:
function get_P_no_prepay(P0,m,t,T)
# Principle without prepayment
    Pt = P0*(1-exp(-m*(T-t)))/(1-exp(-m*T))
    return Pt
end

function get_ho_t(a,b,t,T_asterisk)
# ho(t) deterministic
    res = b*a*max(t,T_asterisk)
    return res
end

function get_h_t(a,b,t,T_asterisk)
    ho_t = get_ho_t(a,b,t,T_asterisk,gamma,k,r)
    res = ho_t + gamma*max(k-r,0.0)
    return res
end

get_P_no_prepay(1.0,0.03,11,10)

-0.11750260390070846

In [4]:
# Model parameters
params = Dict(
# Treasury
            :l0 => -0.127061,
            :ve =>[0.2715618,0.0195524,0.0009720],
            :ka => [5.6772530,0.2520333,0.147],
            :si =>[0.0181427,0.0422960,0.034],
            :x0 => [0.05095958,0.06725220,0.00961570],
# harzard rate process (PSA params)
    # ho(t) parameters
            :a => 0.024,
            :b => 1,
            :T_asterisk => 2.5, # prepayment date,
            :gamma => 1,
            :k => 0.05, # prepayment strike
    )
e_dist = dist.Exponential(1) # exponential distribution about prepayment. Integration of harzard process > e

Distributions.Exponential{Float64}(θ=1.0)

# Simulation

In [5]:
# Let's write a simulation code
num_paths= 5
annual_steps = 12 # steps per year
# num_paths= 500
# annual_steps = 4000 # steps per year
dt = 1/annual_steps
T = 10 # year
num_steps = T * annual_steps

# variables in simulations
num_x = length(params[:x0])
x = zeros(num_paths,num_x,num_steps)
r = zeros(num_paths,num_steps) # r is sum of x
h = zeros(num_paths,num_steps)
Mt = zeros(num_paths,num_steps)
e = zeros(num_paths,num_steps) # random threshold for prepayment
Random.rand!(e_dist,e) # generate e before running simulation since it is simpler and independent
d_B = dist.Normal(0.0,dt)
dW_x = zeros(num_paths,num_x) # Brownian motion for x

# Initialization
current = 1 # current step
# compute initial interest rate
r[:,current] .= params[:l0]
for i in 1:num_x
    x[:,i,current] .= params[:x0][i] # assign initial treasury variable
    r[:,current] .+= params[:x0][i] # initial interest rate
end

# Run Simulation

In [6]:
# run simulation
ve = params[:ve]
ka = params[:ka]
si = params[:si]
# Run simulation at each timestep. compute for the next value
for current in 1:(num_steps-1)
    # simulate treasury parameters
    Random.rand!(d_B,dW_x) # Brownian motions for x
    next = current + 1
    for i in 1:num_x
        global vol
        drift = (ve[i] .- ka[i]*x[:,i,current])*dt
        vol = si[i]*sqrt.(x[:,i,current]).*dW_x[:,i]
        vol += 0.25*si[i]^2*(dW_x[:,i].^2 .- dt) # Milstein scheme
        x[:,i,next] = x[:,i,current] + drift + vol
        x[:,i,next] = max.(0.0, x[:,i,next]) # reflection for CIR process
    end
    # compute harzard rate
    # check prepayment
    # compute value of 
end

In [7]:
max.(0,[1,2,3,4,-5])

5-element Array{Int64,1}:
 1
 2
 3
 4
 0

In [8]:
stats.mean(x[:,1,end])

0.04805572381754385

In [9]:
stats.mean(x[:,2,end])

0.07433351216130213

In [10]:
stats.mean(x[:,3,end])

0.004665779819755079

In [11]:
ve./ka

3-element Array{Float64,1}:
 0.04783330952487057
 0.07757863742608616
 0.006612244897959184

In [12]:
ve

3-element Array{Float64,1}:
 0.2715618
 0.0195524
 0.000972